In [1]:
import ciropt as co
import cvxpy as cp

In [2]:
L_smooth = 1.
mu = 0.00001

Inductance = 1#5.
R = 1.

# solver = "ca"
solver = "ca_canonical"
# solver = "ca_canonical_X"
# solver = "qcqp_sni"
# solver = "bisection_b"

# Ciropt problem

In [3]:
problem = co.douglas_rachford_splitting(mu, L_smooth, R, Inductance)
problem.obj = problem.b + problem.d

bounds = {  #'b': {"ub": 0.1, "lb": 0.1},
            #'h': {"ub": 1, "lb": 1},
            'alpha': {"ub": 1, "lb": 1},
            'beta': {"ub": 1, "lb": 1},}

# res, sol = problem.solve(solver=solver, bounds=bounds, verbose=True)[:2]
res, sol = problem.solve(solver=solver, verbose=True)[:2]
ca_vars = problem.vars

# res = params = {  'b': 10,
#             'h': 1,
#             'd': 0.,
#             'alpha': 0.5,
#             'beta': 0.5 }
# b = problem.solve(solver=solver, freq=1, max_iters=30, params=params, debug=True)
# print(f"{b = }")
# res["b"] = b

dim_G=9, dim_F=8

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:     5362
Number of nonzeros in inequality constraint Jacobian.:      432
Number of nonzeros in Lagrangian Hessian.............:     1090

Total number of variables............................:      114
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      109
Total numbe

In [4]:
res

{'b': 5.8053024864917695e-08,
 'h': 8.949372722801978e-08,
 'd': 6.2546875476749e-08,
 'alpha': -0.47776333558335843,
 'beta': -0.9337892830739776}

# PEP verification

In [5]:
problem = co.douglas_rachford_splitting(mu, L_smooth, R, Inductance, params=res)
diff = problem.solve(verbose=1, solver=cp.MOSEK)
print(f"{diff=}")

(PEPit) Setting up the problem: size of the main PSD matrix: 9x9
(PEPit) Setting up the problem: performance measure is minimum of 1 element(s)
(PEPit) Setting up the problem: Adding initial conditions and general constraints ...
(PEPit) Setting up the problem: initial conditions and general constraints (0 constraint(s) added)
(PEPit) Setting up the problem: interpolation conditions for 2 function(s)
		 function 1 : Adding 12 scalar constraint(s) ...
		 function 1 : 12 scalar constraint(s) added
		 function 2 : Adding 12 scalar constraint(s) ...
		 function 2 : 12 scalar constraint(s) added
(PEPit) Setting up the problem: constraints for 0 function(s)
(PEPit) Compiling SDP
(PEPit) Calling SDP solver


SolverError: Solver 'MOSEK' failed. Try another solver, or solve with verbose=True for more information.

In [ ]:
# Instantiate PEP
problem = PEPit.PEP()
verbose=False
n = 1
L = 1
theta = 1
alpha = 1
# Declare a convex and a smooth convex function.
func1 = problem.declare_function(ConvexFunction)
func2 = problem.declare_function(SmoothConvexFunction, L=L)
# Define the function to optimize as the sum of func1 and func2
func = func1 + func2

# Start by defining its unique optimal point xs = x_* and its function value fs = F(x_*)
xs = func.stationary_point()
fs = func(xs)

# Then define the starting point x0 of the algorithm and its function value f0
x0 = problem.set_initial_point()

# Compute n steps of the Douglas-Rachford splitting starting from x0
x = [x0 for _ in range(n)]
w = [x0 for _ in range(n + 1)]
for i in range(n):
    x[i], _, _ = proximal_step(w[i], func2, alpha)
    y, _, fy = proximal_step(2 * x[i] - w[i], func1, alpha)
    w[i + 1] = w[i] + theta * (y - x[i])

# Set the initial constraint that is the distance between x0 and xs = x_*
problem.set_initial_condition((x[0] - xs) ** 2 <= 1)

# Set the performance metric to the final distance to the optimum in function values
problem.set_performance_metric((func2(y) + fy) - fs)

# Solve the PEP
pepit_verbose = max(verbose, 0)
pepit_tau = problem.solve(verbose=pepit_verbose)

# Compute theoretical guarantee (for comparison) when theta = 1
if theta == 1 and alpha == 1 and L == 1 and 0 < n <= 10:
    pesto_tau = [1 / 4, 0.1273, 0.0838, 0.0627, 0.0501, 0.0417, 0.0357, 0.0313, 0.0278, 0.0250]
    theoretical_tau = pesto_tau[n - 1]

print(theoretical_tau, pepit_tau)